<a href="https://colab.research.google.com/github/iky8039/Prompt-Gen-GUI/blob/main/Prompt_Gen_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import json
import threading
import io

# 1. State Management
if 'my_keywords' not in globals() or not my_keywords:
    my_keywords = ['AI', 'Research', 'Code', 'Python', 'Data Science', 'Machine Learning']

# 2. UI Components
prompt_area = widgets.Textarea(
    placeholder='Generated prompt will appear here...',
    layout=widgets.Layout(width='100%', height='100px', margin='2px')
)

copy_button = widgets.Button(
    icon='copy',
    tooltip='Copy to clipboard',
    button_style='info',
    layout=widgets.Layout(width='60px', height='50px', margin='2px')
)

add_button = widgets.Button(
    icon='plus',
    tooltip='Add keyword(s) from input and prompt area',
    button_style='success',
    layout=widgets.Layout(width='45px', margin='2px')
)

delete_mode = widgets.ToggleButton(
    value=False,
    icon='trash',
    tooltip='Toggle Delete Mode (ON: Click to remove)',
    button_style='danger',
    layout=widgets.Layout(width='45px', margin='2px')
)

save_btn = widgets.Button(
    icon='download',
    tooltip='Download keyword list',
    button_style='primary',
    layout=widgets.Layout(width='45px', margin='2px')
)

load_btn = widgets.FileUpload(
    accept='.json',
    multiple=False,
    description='Upload',
    layout=widgets.Layout(width='90px', margin='2px')
)
load_btn.style.button_color = '#9b59b6'

search_bar = widgets.Text(
    placeholder='Search or type new keyword...',
    layout=widgets.Layout(flex='1 1 auto', margin='2px')
)

keyword_box = widgets.Box(
    layout=widgets.Layout(
        display='flex',
        flex_flow='row wrap',
        border='1px solid #ddd',
        min_height='80px',
        padding='5px',
        margin='5px 0 0 0'
    )
)

# 3. Logic Handlers
def update_keyword_buttons():
    buttons = []
    term = search_bar.value.lower()
    for kw in my_keywords:
        if term in kw.lower():
            btn = widgets.Button(
                description=kw,
                layout=widgets.Layout(width='auto', margin='2px')
            )
            btn.on_click(on_keyword_action)
            buttons.append(btn)
    keyword_box.children = buttons

def on_keyword_action(b):
    global my_keywords
    if delete_mode.value:
        if b.description in my_keywords:
            my_keywords.remove(b.description)
            update_keyword_buttons()
    else:
        current = prompt_area.value.strip()
        prompt_area.value = f"{current}, {b.description}" if current else b.description

def on_add_clicked(b):
    global my_keywords
    # Add from search bar
    new_kw = search_bar.value.strip()
    if new_kw and new_kw not in my_keywords:
        my_keywords.append(new_kw)
        search_bar.value = ''

    # Add from prompt area (parse tags)
    prompt_text = prompt_area.value
    # Split by comma or newline, then trim
    tags = [t.strip() for t in prompt_text.replace('\n', ',').split(',') if t.strip()]
    for t in tags:
        if t not in my_keywords:
            my_keywords.append(t)

    update_keyword_buttons()

def on_copy_clicked(b):
    js_text = json.dumps(prompt_area.value)
    display(Javascript(f"navigator.clipboard.writeText({js_text});"))
    orig_icon = b.icon
    b.icon = 'check'
    threading.Timer(1.5, lambda: setattr(b, 'icon', orig_icon)).start()

def on_save_clicked(b):
    json_str = json.dumps(my_keywords, ensure_ascii=False, indent=4)
    js_content = json.dumps(json_str)
    js_code = f"""
    (function() {{
        const blob = new Blob([{js_content}], {{ type: 'application/json' }});
        const url = URL.createObjectURL(blob);
        const a = document.createElement('a');
        a.href = url;
        a.download = 'keywords.json';
        document.body.appendChild(a);
        a.click();
        setTimeout(() => {{ document.body.removeChild(a); window.URL.revokeObjectURL(url); }}, 0);
    }})();
    """
    display(Javascript(js_code))

def on_upload_change(change):
    global my_keywords
    if not change['new']: return
    try:
        new_val = change['new']
        file_info = list(new_val.values())[0] if isinstance(new_val, dict) else new_val[0]
        content = file_info['content']
        loaded_data = json.loads(content.decode('utf-8'))
        if isinstance(loaded_data, list):
            my_keywords = loaded_data
            update_keyword_buttons()
    except Exception as e: print(f"Upload error: {e}")

# 4. Bind Events
copy_button.on_click(on_copy_clicked)
add_button.on_click(on_add_clicked)
save_btn.on_click(on_save_clicked)
load_btn.observe(on_upload_change, names='value')
search_bar.observe(lambda change: update_keyword_buttons(), names='value')

# 5. Layout
upper_row = widgets.HBox([prompt_area, copy_button], layout=widgets.Layout(align_items='center'))
middle_row = widgets.HBox([search_bar, add_button, delete_mode, save_btn, load_btn], layout=widgets.Layout(align_items='center'))
final_ui = widgets.VBox([
    widgets.HTML("<h4 style='color: #333;'>Interactive Prompt Builder (PC)</h4>"),
    upper_row, middle_row, keyword_box
])

display(final_ui)
update_keyword_buttons()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import json
import threading
import io

# 1. State Initialization
if 'my_keywords' not in globals() or not my_keywords:
    my_keywords = ['AI', 'Research', 'Code', 'Python', 'Data Science', 'Machine Learning']

# 2. UI Components
prompt_output = widgets.Textarea(
    placeholder='Mobile-optimized prompt output...',
    layout=widgets.Layout(width='100%', height='120px', margin='5px 0')
)

keyword_input = widgets.Text(
    placeholder='Enter or search keywords...',
    layout=widgets.Layout(width='100%', height='50px', margin='5px 0')
)

btn_layout = widgets.Layout(height='50px', min_width='80px', flex='1 1 auto', margin='4px')

btn_copy = widgets.Button(description='Copy', button_style='info', icon='copy', layout=btn_layout)
btn_add = widgets.Button(description='Add', button_style='success', icon='plus', layout=btn_layout)
btn_save = widgets.Button(description='Save', button_style='primary', icon='download', layout=btn_layout)
btn_load = widgets.FileUpload(accept='.json', multiple=False, layout=btn_layout, description='Load')

delete_toggle = widgets.ToggleButton(
    value=False,
    description='Delete Mode',
    button_style='danger',
    icon='trash',
    layout=widgets.Layout(height='50px', width='100%', margin='5px 0')
)

keyword_display_box = widgets.Box(
    layout=widgets.Layout(
        display='flex', flex_flow='row wrap', width='100%',
        min_height='100px', border='1px solid #ccc', padding='10px', margin='10px 0'
    )
)

# 3. Logic Handlers
def update_mobile_keyword_display():
    search_term = keyword_input.value.lower()
    filtered = [kw for kw in my_keywords if search_term in kw.lower()]
    btns = []
    for kw in filtered:
        btn = widgets.Button(description=kw, layout=widgets.Layout(height='45px', width='auto', margin='4px'))
        btn.on_click(on_mobile_keyword_action)
        btns.append(btn)
    keyword_display_box.children = btns

def on_mobile_keyword_action(b):
    global my_keywords
    kw = b.description
    if delete_toggle.value:
        if kw in my_keywords:
            my_keywords.remove(kw)
            update_mobile_keyword_display()
    else:
        current = prompt_output.value.strip()
        prompt_output.value = f"{current}, {kw}" if current else kw

def on_mobile_add_clicked(b):
    global my_keywords
    # 1. Add from input field
    new_val = keyword_input.value.strip()
    if new_val and new_val not in my_keywords:
        my_keywords.append(new_val)
        keyword_input.value = ''

    # 2. Add from prompt area (extract unique keywords)
    prompt_text = prompt_output.value
    tags = [t.strip() for t in prompt_text.replace('\n', ',').split(',') if t.strip()]
    for t in tags:
        if t not in my_keywords:
            my_keywords.append(t)

    update_mobile_keyword_display()

def on_mobile_copy_clicked(b):
    js_code = f"navigator.clipboard.writeText({json.dumps(prompt_output.value)});"
    display(Javascript(js_code))
    old_icon = b.icon
    b.icon = 'check'
    threading.Timer(1.0, lambda: setattr(b, 'icon', old_icon)).start()

def on_mobile_save_clicked(b):
    json_str = json.dumps(my_keywords, ensure_ascii=False, indent=4)
    js_content = json.dumps(json_str)
    js_code = f"""
    (function() {{
        const blob = new Blob([{js_content}], {{ type: 'application/json' }});
        const url = URL.createObjectURL(blob);
        const a = document.createElement('a');
        a.href = url; a.download = 'keywords.json';
        document.body.appendChild(a); a.click();
        setTimeout(() => {{ document.body.removeChild(a); window.URL.revokeObjectURL(url); }}, 0);
    }})();
    """
    display(Javascript(js_code))

def on_mobile_load_change(change):
    global my_keywords
    if not change['new']: return
    try:
        new_val = change['new']
        file_info = list(new_val.values())[0] if isinstance(new_val, dict) else new_val[0]
        content = file_info['content']
        loaded_data = json.loads(content.decode('utf-8'))
        if isinstance(loaded_data, list):
            my_keywords = loaded_data
            update_mobile_keyword_display()
    except Exception as e: print(f"Error: {e}")

def on_delete_toggle_change(change):
    if change['new']:
        delete_toggle.description = 'Tap to REMOVE'
        delete_toggle.button_style = 'warning'
    else:
        delete_toggle.description = 'Delete Mode'
        delete_toggle.button_style = 'danger'

# 4. Bind Events
btn_add.on_click(on_mobile_add_clicked)
btn_copy.on_click(on_mobile_copy_clicked)
btn_save.on_click(on_mobile_save_clicked)
btn_load.observe(on_mobile_load_change, names='value')
keyword_input.observe(lambda change: update_mobile_keyword_display(), names='value')
delete_toggle.observe(on_delete_toggle_change, names='value')

# 5. Assemble
mobile_ui = widgets.VBox([
    widgets.HTML("<h2 style='text-align: center; color: #444;'>Mobile Prompt Builder</h2>"),
    prompt_output,
    widgets.HBox([btn_copy, btn_save, btn_load], layout=widgets.Layout(width='100%')),
    widgets.HTML("<b>Manage Keywords</b>"),
    keyword_input,
    btn_add,
    delete_toggle,
    keyword_display_box
], layout=widgets.Layout(width='100%', max_width='600px', margin='0 auto', padding='10px'))

display(mobile_ui)
update_mobile_keyword_display()